In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

df = pd.read_csv('../raw_data/prob_of_first_vaccination.csv')
vac_prob = pd.read_csv('../raw_data/Marshfield_vaccination_timing.csv')
new_rows = []
for vactype, vactypedf in vac_prob.groupby('Vaccine type'):
    temp = vactypedf.copy()
    temp['cdf'] = np.cumsum(temp.PMF)
    for index, row in temp.iterrows():
        if row.cdf >= 0.5:
            year, week = (row.Year, row.Week)
            new_rows.append([vactype, year, week])
            break
            
median_weeks = pd.DataFrame(new_rows, columns = ['vactype', 'year', 'week'])
median_weeks.index = median_weeks.vactype
vac_median = median_weeks.median().week
vactypes = [str(vactype) for vactype in range(1918, 2010)] + ['2009Pan'] + [str(vactype) for vactype in range(2010, 2019)]
new_rows = []
for birth_year in range(1918, 2018):
    for vactype in vactypes:
        if vactype in list(median_weeks.vactype):
            median = median_weeks.loc[vactype, 'week']
            year = median_weeks.loc[vactype, 'year']
        else:
            median = vac_median
            year = int(vactype)
        
        if birth_year <= year:
            a1 = year - birth_year
            a2 = a1 - 1
            f1 = median / 52
            f2 = 1 - f1
            
            if a1 == 0:
                p = df.loc[0, 'probability_first_vaccine']
            elif a1 <= 13:
                p1 = df.loc[a1, 'probability_first_vaccine']
                p2 = df.loc[a2, 'probability_first_vaccine']
                p = p1 * f1 + p2 * f2
            else:
                p = 0
            new_rows.append([birth_year, vactype, p])
outdf = pd.DataFrame(new_rows, columns=['Birth_year', 'vactype', 'Naive_vac'])
outdf.to_csv('../data/Marshfield_naive_vac_by_birth_year.csv', index=False)